In [ ]:
!wget -O - --no-check-certificate https://hyeshik.qbio.io/binfo/binfo1-datapack1.tar | tar -C /mnt/d/binfodata -xf -
!wget --no-check-certificate -O /mnt/d/binfodata/binfo1-datapack1/gencode.gtf.gz http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M27/gencode.vM27.annotation.gtf.gz
!md5sum -c /mnt/d/binfodata/binfo1-datapack1/*
!ls -al /mnt/d/binfodata/binfo*

In [ ]:
!mkdir -p /mnt/d/binfodata/binfo1-work/
%cd /mnt/d/binfodata/binfo1-work/
!cp ../binfo1-datapack1/* .

In [89]:
!conda install -y bedtools bioawk samtools subread

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: / warning  libmamba Added empty dependency for problem type SOLVER_RULE_UPDATE
| ^C


In [90]:
!pip install pysam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.1/25.1 MB 10.1 MB/s eta 0:00:0000:0100:01


압축풀기(reference genome)

In [84]:
!gunzip /mnt/d/binfodata/binfo1-datapack1/mm9.fa.gz

Directory 지정

In [2]:
%cd /mnt/d/binfodata/binfo1-work

/mnt/d/binfodata/binfo1-work


In [64]:
!grep ' .*transcript_support_level "1"' ../binfo1-datapack1/gencode.gtf | \
 sed -e 's/	[^	]*transcript_id "\([^"]*\)".*$/	\1/g' > gencode-tsl1.gtf


chr1	HAVANA	transcript	3276124	3286567	.	-	.	ENSMUST00000162897.2
chr1	HAVANA	exon	3283832	3286567	.	-	.	ENSMUST00000162897.2
chr1	HAVANA	exon	3276124	3277540	.	-	.	ENSMUST00000162897.2
chr1	HAVANA	transcript	3276746	3285855	.	-	.	ENSMUST00000159265.2
chr1	HAVANA	exon	3283662	3285855	.	-	.	ENSMUST00000159265.2
chr1	HAVANA	exon	3276746	3277540	.	-	.	ENSMUST00000159265.2
chr1	HAVANA	transcript	3284705	3741721	.	-	.	ENSMUST00000070533.5
chr1	HAVANA	exon	3740775	3741721	.	-	.	ENSMUST00000070533.5
chr1	HAVANA	CDS	3740775	3741571	.	-	0	ENSMUST00000070533.5
chr1	HAVANA	start_codon	3741569	3741571	.	-	0	ENSMUST00000070533.5


In [75]:
!tail gencode-tsl1.gtf

chrY	HAVANA	exon	90614770	90614838	.	-	.	ENSMUST00000189170.7
chrY	HAVANA	transcript	90614770	90617133	.	-	.	ENSMUST00000185955.2
chrY	HAVANA	exon	90616948	90617133	.	-	.	ENSMUST00000185955.2
chrY	HAVANA	exon	90616279	90616460	.	-	.	ENSMUST00000185955.2
chrY	HAVANA	exon	90614770	90614838	.	-	.	ENSMUST00000185955.2
chrY	HAVANA	transcript	90796007	90827716	.	+	.	ENSMUST00000179483.9
chrY	HAVANA	exon	90796007	90796091	.	+	.	ENSMUST00000179483.9
chrY	HAVANA	exon	90796859	90797248	.	+	.	ENSMUST00000179483.9
chrY	HAVANA	exon	90804565	90804686	.	+	.	ENSMUST00000179483.9
chrY	HAVANA	exon	90827618	90827716	.	+	.	ENSMUST00000179483.9


Samtools를 이용해 bam 파일의 unmapped data를 제거합니다.

In [ ]:
!(samtools view -H ../binfo1-datapack1/CLIP-35L33G.bam; \
  samtools view -F 4 ../binfo1-datapack1/CLIP-35L33G.bam) | \
 samtools view -b -o filtered-CLIP-35L33G.bam

In [92]:
!samtools mpileup filtered-CLIP-35L33G.bam > filtered-CLIP-35L33G-pileup.bam

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


In [81]:
import pandas as pd

# read counts 데이터 읽기
cnts = pd.read_csv('read-counts.txt', sep='\t', comment='#', index_col=0)

# CLIP-35L33G.bam 열의 read depth가 50 이상인 행만 필터링
filtered_cnts = cnts[cnts['filtered-CLIP-35L33G.bam'] >= 50]

# 필요하다면, 결과를 새로운 파일에 저장
filtered_cnts.to_csv('50filter_read_counts.txt', sep='\t')

In [82]:
filtered_cnts.head()

,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-CLIP-35L33G.bam,filtered-RPF-siLuc.bam,filtered_CLIP-35L33G.bam
Geneid,,,,,,,,,,,,,,,
ENSMUSG00000102343.2,chr1;chr1;chr1;chr1;chr1,3975962;4054448;4055383;4055383;4056370,3976357;4054521;4056207;4055574;4056438,-;-;-;-;-,1364,67,0,31,207,157,1,2,67,0,0
ENSMUSG00000102948.2,chr1;chr1,4326457;4329070,4326650;4330742,-;-,1867,60,0,1,1,2,0,0,60,0,0
ENSMUSG00000103922.2,chr1,4841354,4842422,+,1069,183,0,95,93,100,42,75,183,74,0
ENSMUSG00000033845.14,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4843429;4843434;4844659;4846600;4847748;484774...,4844739;4847024;4844739;4847024;4847871;484787...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,8487,4014,0,1355,3022,2563,1859,3191,4014,3,0
ENSMUSG00000025903.15,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4878011;4878046;4878053;4878119;4878121;487813...,4878205;4878205;4878205;4878205;4878205;487820...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;...,7145,2045,0,574,1219,1236,33,44,2045,44,0


In [85]:
import pandas as pd
import subprocess

# Step 1: read_counts 파일에서 유전자 위치 추출 및 BED 파일 생성
read_counts_file = '50filter_read_counts.txt'
output_bed_file = 'filtered_gene_coordinates.bed'

# read_counts 파일을 pandas DataFrame으로 읽기
read_counts = pd.read_csv(read_counts_file, sep='\t')

# 유전자 위치 추출 및 BED 형식으로 저장
with open(output_bed_file, 'w') as bed_file:
    for index, row in read_counts.iterrows():
        bed_file.write(f"{row['Chr']}\t{row['Start']}\t{row['End']}\t{row['Geneid']}\n")

# Step 2: samtools mpileup을 사용하여 유전자 위치에 대한 pileup 수행
reference_genome = '/mnt/d/binfodata/binfo1-datapack1/mm9.fa'  # reference genome의 경로를 지정하세요
bam_file = 'filtered-CLIP-35L33G.bam'
output_pileup_file = 'pileup_output.txt'

mpileup_command = f"samtools mpileup -f {reference_genome} -l {output_bed_file} {bam_file} > {output_pileup_file}"
subprocess.run(mpileup_command, shell=True, check=True)

print(f"Pileup output saved as: {output_pileup_file}")

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000
[mplp_func] Skipping because 156075375 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075375 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075375 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075484 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075484 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075484 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075484 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075484 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075510 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075510 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075510 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075510 is outside of 155630120 [ref:3]
[mplp_func] Skipping because 156075510 is outside of 155630120 [ref:3]

Pileup output saved as: pileup_output.txt


In [87]:
!tail pileup_output.txt

chrM	15413	C	83	,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,	DBB=EIIIEHI;HIHI2FIIHIIGIGGGEIIGI@IIIGABGGHHBIG<IIHHGIGHIEDB0GIFIIGIIIIBII;IIGIIBIH
chrM	15414	T	83	,,$,,,a,,,,,,,$,,$,,a,,,,,,a,,a,,,$a,$aaa,,,,aaaaaaaaaaaaaaaaaaa,,,,,,,,,,,,,,,,aaaaaaaa	:CBCEHIIDII;DIDI=DIIHIIGIGGBDIDGD?IIIGBCGEIHAIGEIIHHGIGHIGG@9DIGIIGIIII?II4IIGHICIH
chrM	15415	A	76	,,g,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,	8B/EHIDAIIHI=CHHFIIDICDGFIG@IIIB?:GDHH?IG?IIHHGIGHHGGG=GIGII>IIII:IHGIDII>IH
chrM	15416	C	78	,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,	DDC?HDIDII6II6FIIHIIDIEGEFIG=IIIGBCGGHHBIGEGIHHGIGHIGE58GIFIIBIIIIBIH7IIGIIBIH
chrM	15417	T	78	,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,	=B?EIIIFII6II8GIGHIIGIEGGEHG:IIIGCADBGHEIBDEGHHHIHGI?>BBGIHGIDIIIIDIH7IIBIIBIH
chrM	15418	T	78	,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,	G3E4III@II=IH?

In [91]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pysam

def calculate_shannon_entropy(counts):
    total = sum(counts)
    if total == 0:
        return 0
    probs = [count / total for count in counts]
    return -sum(p * np.log2(p) for p in probs if p > 0)

def calculate_cres(entropies, window_size=5):
    half_window = window_size // 2
    cres_values = []
    for i in range(len(entropies)):
        window = entropies[max(0, i - half_window): min(len(entropies), i + half_window + 1)]
        cres_values.append(np.mean(window))
    return cres_values

# Step 1: Shannon entropy 및 CRES 값 계산
pileup_file = 'pileup_output.txt'
cutoff = 0.8

pileup_data = defaultdict(list)
with open(pileup_file, 'r') as f:
    for line in f:
        chrom, pos, ref, depth, bases, qual = line.strip().split('\t')[:6]
        base_counts = [bases.count(base) for base in 'ACGT']
        entropy = calculate_shannon_entropy(base_counts)
        pileup_data['chrom'].append(chrom)
        pileup_data['pos'].append(int(pos))
        pileup_data['entropy'].append(entropy)

pileup_df = pd.DataFrame(pileup_data)
pileup_df['cres'] = calculate_cres(pileup_df['entropy'].tolist())

# CRES 값이 cutoff를 통과하는 binding site 선택
binding_sites = pileup_df[pileup_df['cres'] > cutoff]
binding_sites.to_csv('binding_sites.csv', index=False)